# Installation packages

In [35]:
!pip install pydicom
!pip install dicom2jpg
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [2]:
from mycode import convert_images_to_vgg, convert_coco_to_vgg, convert_vgg_to_images, nii2images, nifti2dicom_1file
from PIL import Image

import nibabel as nib
import numpy as np
import SimpleITK as sitk

import dicom2jpg
import pathlib
import glob
import cv2
import os

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Main part

## Convert masks to VGG

In [3]:
# Создание фото масок
filename_masks = pathlib.Path('./data/00000057_final_seg.nii')
output_path_masks = pathlib.Path('./data/mask2coco/masks')

nii2images(filename=filename_masks, output_path=output_path_masks, mask_flag=True)

In [4]:
# Создание фото исходных изображений
filename_imgs = pathlib.Path('./data/00000057_brain_flair.nii')
output_path_imgs = pathlib.Path('./data/mask2coco/im')

nii2images(filename=filename_imgs, output_path=output_path_imgs)

In [5]:
# Создание json-файла в формате vgg на основе масок и исходных изображений
convert_images_to_vgg(output_json=output_path_imgs / 'vgg.json', 
                      mask_path=output_path_masks,
                      img_dir=output_path_imgs,
                      tolerance=0.9)

  0%|          | 0/155 [00:00<?, ?it/s]

## Convert vgg to images

In [6]:
# Создание масок из json-файла, полученного после разметки/доразметки
ann_paths = [pathlib.Path('./data/mask2coco/im/vgg.json')]
output_path = pathlib.Path('./data/mask2coco/masks_gen')
photos_path = pathlib.Path('./data/mask2coco/im')

convert_vgg_to_images(ann_paths=ann_paths,
                      output_path=output_path,
                      photos_path = photos_path)

  0%|          | 0/155 [00:00<?, ?it/s]

## Convert images to nifti

In [7]:
# Преобразование изображений в nifti
output_path = pathlib.Path('./data/mask2coco/masks_gen')
source = nib.load(filename='./data/00000057_final_seg.nii')

new_data = np.array([np.array(Image.open(fname)) for fname in sorted(glob.glob(str(output_path / '*.png')))])

new_nifti = nib.Nifti1Image(np.transpose(new_data, (1, 2, 0)), source.affine, source.header)
nib.save(new_nifti, './data/new_nifti.nii')

## Convert nifti to dicom

In [8]:
# Преобразование nifti в серию dicom
nifti2dicom_1file('./data/new_nifti.nii', './data/nifti2dicom')

  0%|          | 0/155 [00:00<?, ?it/s]

## Convert dicom to images

In [9]:
# Перобразование dicom в изображения
dicom_dir = pathlib.Path('./data/nifti2dicom')
dicom_postfix = False

In [10]:
if dicom_postfix: # если dicom файлы представлены без расширения
    for i in os.listdir(dicom_dir):
        os.rename(dicom_dir / i, dicom_dir / (i + '.dcm'))

for n, dcm in enumerate(sorted(glob.glob(str(dicom_dir / '*')))):
    temp = dicom2jpg.dicom2img(dcm)
    new_path = dicom_dir.parent / (dicom_dir.name + "_jpg")
    os.makedirs(new_path, exist_ok=True)
    cv2.imwrite(str(new_path / f'{str(n).zfill(5)}.jpg'), temp)

c:\Users\ReeD\anaconda3\lib\site-packages\dicom2jpg\utils.py:120: RuntimeWarning: invalid value encountered in divide
  pixel_array = ((pixel_array-pixel_array.min())/(pixel_array.max()-pixel_array.min())) * 255.0


## Convert dicom to nifti

In [11]:
# Преобразование серии dicom в формат nifti
dcm_path = './data/nifti2dicom/'
filename_out = 'data/dcm_to_nifti.nii'

reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(dcm_path)
reader.SetFileNames(dicom_names)
image = reader.Execute()

sitk.WriteImage(image, filename_out)